In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import datetime
import numpy as np
import seaborn as sns
import re
import os
import sys
sys.path.append('..')
from utilities import logger
from utilities import configuration


In [2]:
config = configuration.get_config()
for key in config:
    print(f'{key:40} (type: {type(config[key])})')


logging = logger.init_logger(config['system_log'])
logging.debug('Logger has started ont notebook 05. Analysing features.ipynb ...')

system_log                               (type: <class 'str'>)
unified_merged_file_cz                   (type: <class 'str'>)
unified_merged_file_noncz                (type: <class 'str'>)
unified_merged_file                      (type: <class 'str'>)
data_path                                (type: <class 'str'>)
cz_files                                 (type: <class 'list'>)
noncz_files                              (type: <class 'list'>)
2023-09-27 11:42:33,850 - root - DEBUG - Logger has started ont notebook 05. Analysing features.ipynb ...


In [3]:
df = pd.read_csv(config['unified_merged_file'])
df.shape

/var/folders/d0/9279s72s163gx0_12qbmr9440000gn/T/ipykernel_5546/852065790.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config['unified_merged_file'])


(618697, 31)

In [9]:
print(f"ALC patient count: {np.sum(df['ALCDays']>0):,}")
print(f"Percentage = {100*(np.sum(df['ALCDays']>0)/df.shape[0]):.2f} %")

ALC patient count: 22,230
Percentage = 3.59 %


In [14]:
len(set(df['Coded HCN']))

442263

In [6]:
df.iloc[0,:]

Institution Number:                                                        65
Coded HCN                                                           1159480.0
Admit Date:                                                        2014-12-24
Disch Date:                                                        2015-01-01
Readmission Code:                        5 New patient to the acute care unit
Patient Age:                                                               67
Gender:                                                                  Male
MRDx                           K746  Other and unspecified cirrhosis of liver
Postal Code:                                                           B3A2K1
Diagnosis:                  ['K746', 'K431', 'R18', 'K760', 'K650', 'I518'...
Diagnosis Long Text         ['Other and unspecified cirrhosis of liver', '...
Diagnosis Type              ['M', '1', 'W', '1', '1', '1', '1', '1', '1', ...
Intervention Code                                               

In [26]:
df.sort_values(by='Total Days in Hospital', ascending=False).iloc[:1000,:].to_csv(os.path.join(config['data_path'], 'by_leng_of_stay.csv'))
df.sort_values(by='ALCDays', ascending=False).iloc[:1000,:].to_csv(os.path.join(config['data_path'], 'by_ALCDays.csv'))
df.sort_values(by='Acute Days', ascending=False).iloc[:1000,:].to_csv(os.path.join(config['data_path'], 'by_Acute_Days.csv'))

In [58]:
ix=0
while ix<df.shape[0] and df['Diagnosis:'].iloc[ix]!='**':
    ix+=1

ix

1730

In [59]:
df.iloc[1730,:]

Institution Number:                                           85
Coded HCN                                             40211217.0
Admit Date:                                           2015-01-01
Disch Date:                                           2015-01-05
Readmission Code:           5 New patient to the acute care unit
Patient Age:                                                  30
Gender:                                                     Male
MRDx                                                            
Postal Code:                                              B0N2T0
Diagnosis:                                                    **
Diagnosis Long Text                                           **
Diagnosis Type                                                **
Intervention Code                                             **
Px Long Text                                                  **
Admit Category:                               U Emergent, urgent
Entry Code:              

In [82]:
list(df['Diagnosis Type'].iloc[:3])

["['M', '1', 'W', '1', '1', '1', '1', '1', '1', '3']",
 "['M', '3']",
 "['M', '3', '9']"]

In [75]:
for row_ix in range(df.shape[0]):
    row = df.iloc[row_ix,:]
    if row['Diagnosis:'] == '**':
        diagnosis_codes = []
    elif '[' in row['Diagnosis:'] and ']' in row['Diagnosis:'] :       
        diagnosis_codes = re.findall("'(.*?)'", row['Diagnosis:'] )
    else:
        diagnosis_codes = [row['Diagnosis:']]

    if row['MRDx'].strip()=='':
        assert diagnosis_codes==[]
    else:
        assert row['MRDx'].startswith(diagnosis_codes[0])
    # assert df['MRDx'].iloc[row_ix].startswith()
    



In [47]:
df.drop(columns=['Institution Number:',
                 'Coded HCN',
                 'Postal Code:',
                 'Diagnosis:',
                 'Diagnosis Type',
                 'Intervention Code',
                 'Institution From',
                 'Institution To',
                 'Institution Type',
                 'Discharge Nurse Unit',
                 'Disch Date (year-month):',
                 'is ALC Patient',
                 'Total Days in Hospital'
                 ]).shape

(618697, 18)

In [45]:
values=[]
for value in df['Case Weight']:
    if value!='**':
        if isinstance(value, str):
            value = float(value.replace(',', ''))
        values.append(value)
        
values = np.array([values])
np.average(values[~np.isnan(values)])

1.8073578002770827

In [7]:
set(df['Coded HCN'].iloc[:10])

0       1159480.0
1       2232961.0
2       9938495.0
3      12350498.0
4    8000546788.0
5       8419505.0
6       7264068.0
7       3605266.0
8       8321248.0
9       7444249.0
Name: Coded HCN, dtype: object

Discards obviously irrelevant, see what is left.
